In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
from imutils import paths
import os

In [2]:
dataset_dir = "../data/images/FaceRec"
train_dir = os.path.join(dataset_dir,"trainFaces")

train_imagepaths = list(paths.list_images(train_dir))
train_labels = [x.split(os.path.sep)[-2] for x in train_imagepaths]

In [3]:
label_mapping = {'vikas': 0,'vaibhaw':1,'satya':2,'Jash':3,'koustubh':4}
inv_label_mapping = dict([(v,k) for k,v in label_mapping.items()])
inv_label_mapping

{0: 'vikas', 1: 'vaibhaw', 2: 'satya', 3: 'Jash', 4: 'koustubh'}

In [5]:
def align_face(image,landmarks):
    width = 600
    height = 600
    eye_points_src = [[int(landmarks.part(2).x),int(landmarks.part(2).y)],[int(landmarks.part(0).x),int(landmarks.part(0).y)]]
    eye_points_dst = [[int(0.3 * width),int(height/3)],[int(0.7*width),int(height/3)]]

    cos_theta = np.cos(60*np.pi/180)
    sin_theta = np.sin(60*np.pi/180)

    x_src = cos_theta*(eye_points_src[0][0] - eye_points_src[1][0]) - sin_theta*(eye_points_src[0][1] - eye_points_src[1][1]) + eye_points_src[1][0]
    y_src = cos_theta*(eye_points_src[0][0] - eye_points_src[1][0]) + sin_theta*(eye_points_src[0][1] - eye_points_src[1][1]) + eye_points_src[1][1]

    x_dst = cos_theta*(eye_points_dst[0][0] - eye_points_dst[1][0]) - sin_theta*(eye_points_dst[0][1] - eye_points_dst[1][1]) + eye_points_dst[1][0]
    y_dst = cos_theta*(eye_points_dst[0][0] - eye_points_dst[1][0]) + sin_theta*(eye_points_dst[0][1] - eye_points_dst[1][1]) + eye_points_dst[1][1]

    eye_points_src.append([x_src,y_src])
    eye_points_dst.append([x_dst,y_dst])

    eye_points_src = np.int32(eye_points_src).reshape(3,1,2)
    eye_points_dst = np.int32(eye_points_dst).reshape(3,1,2)

    transform_mat = cv2.estimateAffinePartial2D(eye_points_src,eye_points_dst)[0]

    face_aligned = cv2.warpAffine(image,transform_mat,(width,height))
    
    return face_aligned


In [4]:
face_detector = dlib.get_frontal_face_detector()
landmark_detector = dlib.shape_predictor("../data/models/shape_predictor_5_face_landmarks.dat")

In [6]:
rec_model = cv2.dnn.readNetFromTorch("../data/models/openface.nn4.small2.v1.t7")
rec_mean = [0,0,0]
rec_size = (96, 96)
rec_scale = 1/255.0

In [8]:
face_descriptors = []
labels = []
for (imagepath,label) in zip(train_imagepaths,train_labels):
    image = cv2.imread(imagepath)
    
    image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    face_rects = face_detector(image_rgb,0)
    
    if len(face_rects) == 0:
        continue
        
    x1 = int(face_rects[0].left())
    y1 = int(face_rects[0].top())
    x2 = int(face_rects[0].right())
    y2 = int(face_rects[0].bottom())
    
    new_rect = dlib.rectangle(x1,y1,x2,y2)
    
    landmarks = landmark_detector(image_rgb,new_rect)
    
    face_aligned = align_face(image,landmarks)
    
    blob = cv2.dnn.blobFromImage(face_aligned,rec_scale,rec_size,rec_mean,False,False)
    rec_model.setInput(blob)
    face_descriptor = rec_model.forward()
    
    face_descriptors.append(face_descriptor)
    labels.append(label_mapping[label])

In [12]:
face_descriptors = np.vstack(face_descriptors)

In [24]:
cap = cv2.VideoCapture(0)

In [25]:
while True:
    starting_time = cv2.getTickCount()
    ret,frame = cap.read()
    
    if not ret:
        break
        
    frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    face_rects = face_detector(frame_rgb,0)
    
    if len(face_rects) == 0:
        continue
        
    x1 = int(face_rects[0].left())
    y1 = int(face_rects[0].top())
    x2 = int(face_rects[0].right())
    y2 = int(face_rects[0].bottom())
    
    new_rect = dlib.rectangle(x1,y1,x2,y2)
    
    landmarks = landmark_detector(frame_rgb,new_rect)
    
    face_aligned = align_face(frame,landmarks)
    
    blob = cv2.dnn.blobFromImage(face_aligned,rec_scale,rec_size,rec_mean,False,False)
    rec_model.setInput(blob)
    face_descriptor = rec_model.forward()
    
    distances = np.linalg.norm((face_descriptors - face_descriptor),axis = 1)
    
    index = np.argmin(distances)
    distance = distances[index]
    
    if distance < 0.8:
        person_name = inv_label_mapping[labels[index]]
    else:
        person_name = "unknown"
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount() - starting_time)
    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0))
    cv2.putText(frame,"Name : {} FPS : {:.3f}".format(person_name,fps),(10,25),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255))
    
    cv2.imshow("frame",frame)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
        
    
cap.release()
cv2.destroyAllWindows()